# Prediction
After the exploratory data analysis, a prediction will be made.
The folowing steps for this prediction:
1. Preprocessing - Handle missing data, Feature engineering.
2. Predictions - predicting with validation data and then with test data, metrics.
## 📚

In [194]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

#pipeline tools
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from pipelinehelper import PipelineHelper

## Baseline 1: Basic approach

### Decisions:
* **Preprocessing:**
    1. **Missing data:** **Categoric** - fill with most frequent. **Discrete** - fill with most frequent. **Continuous** - fill with median.               
    2. **Feature Engineering**: only categoric labels, label encoding.
* **model training** - using cross validation in validation data on a simple decition tree model.
* **model testing** - train on whole train + validation set and use test data for results.

In [195]:
cat_imp = SimpleImputer(strategy="most_frequent")

Categoric = FunctionTransformer(lambda x: x[Cats], validate=False)
Categoric_features = Pipeline([
                ('selector', Categoric),
                ('imputer', cat_imp),
                ('LabelEncoder', LabelEncoder())
            ])

In [197]:
cont_imp = SimpleImputer(strategy="median")
Continuous = Continuous+["LOAN"]
Cont_FT = FunctionTransformer(lambda x: x[Continuous], validate=False)
Continuous_features = Pipeline([
                ('selector', Cont_FT),
                ('imputer', cont_imp)
            ])